In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 로드

In [ ]:
genre_gn_all = pd.read_json('C:/Users/kjh96/Github/arena/genre_gn_all.json', typ = 'series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
genre_gn_all.head(3)

# 1. 곡 장르 코드 데이터
### 대분류 장르코드

In [3]:
# 장르코드 뒷자리 두 자리가 00인 코드를 필터링
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']

### 상세 장르코드

In [4]:
# 장르코드 뒷자리 두 자리가 00이 아닌 코드를 필터링
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)

C:\Users\kjh96\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### 장르 코드 트리

In [5]:
# 앞자리 네 자리 공통코드 추출
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# Merge
gnr_code_tree = pd.merge(gnr_code, dtl_gnr_code, how = 'left', on = 'join_code')

# 2. 노래 메타 데이터

In [7]:
song_meta = pd.read_json('C:/Users/kjh96/Github/arena/song_meta.json')

# 3. 플레이리스트 Train data

In [34]:
train = pd.read_json('C:/Users/kjh96/Github/arena/train.json')
# genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
# genre_gn_all.head(3)
train = train[:200]
train_df = train[['plylst_title','songs','tags']]

In [35]:
list_df = []
for i in range(len(train_df)) :
    df = pd.DataFrame(train_df['songs'][i])
    df = df.assign(tags = " ".join(train_df['tags'][i]),plylst_title = train_df['plylst_title'][i])
    list_df.append(df)

In [36]:
for i in range(len(list_df)) : 
    if i == 0 :    
        df_fin = list_df[0]
    else :
        df_fin = pd.concat([df_fin,list_df[i]])

In [37]:
df_fin = df_fin.rename(columns={0 : 'song_id'})

In [38]:
song_meta = song_meta.rename(columns={'id' : 'song_id'})

In [39]:
# 노래 메타 정보와 merge
df_fin = pd.merge(df_fin, song_meta, how = 'left', on = "song_id")

# 콘텐츠 기반 필터링 추천(Content based filtering)

In [45]:
data = df_fin

In [46]:
data

,song_id,tags,plylst_title,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
0,525514,락,여행같은 음악,"[GN1402, GN1401]",20130506,Hey Little Girl,2200223,[734201],Hey Little Girl,[GN1400],[The Sol]
1,129701,락,여행같은 음악,"[GN0901, GN0902, GN1001]",20130917,Brass,2201802,[536907],Octagon,"[GN0900, GN1000]",[Royal Bangs]
2,383374,락,여행같은 음악,"[GN1012, GN1005, GN1001]",19911021,Monsters Under The Bed,2216938,[166978],The Road,[GN1000],[Honeymoon Suite]
3,562083,락,여행같은 음악,"[GN1013, GN0901, GN0902, GN1001]",20000919,United,43227,[19035],Honeymoon,"[GN0900, GN1000]",[Phoenix]
4,297861,락,여행같은 음악,"[GN1013, GN0901, GN0902, GN1001]",20050306,Back To Bedlam,303657,[170117],High,"[GN0900, GN1000]",[James Blunt]
...,...,...,...,...,...,...,...,...,...,...,...
8033,37212,힙합 감성 나만알고싶은 dope Pop BlackMusic HipHop 기분전환 내...,도입부부터 느낌오는 : Mad Sexy Hiphop Playlist,"[GN1201, GN1202]",20170620,No Fear,10072619,[788363],No Fear,[GN1200],[Dej Loaf]
8034,324322,힙합 감성 나만알고싶은 dope Pop BlackMusic HipHop 기분전환 내...,도입부부터 느낌오는 : Mad Sexy Hiphop Playlist,"[GN1304, GN1301, GN1302]",20170825,Sky Walker (Feat. Travis Scott),10090628,[553325],Sky Walker (Feat. Travis Scott),[GN1300],[Miguel]
8035,14430,힙합 감성 나만알고싶은 dope Pop BlackMusic HipHop 기분전환 내...,도입부부터 느낌오는 : Mad Sexy Hiphop Playlist,[GN1201],20171103,Without Warning,10108103,"[991191, 994217, 1813840]",Ric Flair Drip,[GN1200],"[21 Savage, Offset, Metro Boomin]"
8036,149760,힙합 감성 나만알고싶은 dope Pop BlackMusic HipHop 기분전환 내...,도입부부터 느낌오는 : Mad Sexy Hiphop Playlist,[GN1301],20171215,The Weekend (Funk Wav Remix),10120108,"[992198, 227229]",The Weekend (Funk Wav Remix),[GN1300],"[SZA, Calvin Harris]"


In [48]:
count_vector = CountVectorizer(ngram_range=(1, 1))
c_vector_genres = count_vector.fit_transform(data['album_name'])
c_vector_genres.shape

(8038, 6329)

In [49]:
#코사인 유사도를 구한 벡터를 미리 저장
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]
gerne_c_sim.shape
gerne_c_sim

array([[   0, 3496, 4058, ..., 5350, 5351, 4018],
       [   1, 7147, 8037, ..., 5364, 5365,    0],
       [   2, 1919, 3520, ..., 5229, 5232,    0],
       ...,
       [8035, 7551,  443, ..., 5363, 5364,    0],
       [8036, 1797, 4778, ..., 5237, 5238,    0],
       [8037, 2616, 3564, ..., 5350, 5351,    0]], dtype=int64)

In [50]:
def get_recommend_movie_list(df, song_id, top=10):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_movie_index = df[df['song_id'] == song_id].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = sim_index
    return result

In [51]:
get_recommend_movie_list(data, song_id= 6546)

C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


array([[2200, 1300,   44,   43, 2488, 5713,   51, 4262, 6270, 6511, 2200,
        1300,   44,   43, 2488, 5713,   51, 4262, 6270, 6511, 2200, 1300,
          44,   43, 2488, 5713,   51, 4262, 6270, 6511]], dtype=int64)

In [31]:
song_meta[song_meta['song_id']==6546]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,song_id
6546,"[GN0105, GN1501, GN0101, GN1504]",20140902,연애의 발견 OST Part 4,2279896,[192827],"묘해, 너와","[GN1500, GN0100]",[어쿠스틱 콜라보],6546


In [55]:
song_meta[song_meta['song_id']== 2488]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,song_id
2488,"[GN1701, GN1706]",20151028,A NIGHT IN SOUTH BLUE MOUNTAIN,2647134,[518540],Montara,[GN1700],[Soil & `Pimp` Sessions],2488
